# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,14.52,66,57,4.46,NZ,1735530924
1,1,christchurch,-43.5333,172.6333,16.11,76,100,10.29,NZ,1735530925
2,2,punta arenas,-53.1500,-70.9167,14.06,72,0,3.60,CL,1735530644
3,3,drahichyn,52.1874,25.1597,-0.78,83,14,4.58,BY,1735530927
4,4,mata-utu,-13.2816,-176.1745,28.58,72,96,3.32,WF,1735530929


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 30 degrees but higher than 25.
# Wind speed less than 10 mph.
# Zero cloudiness.

Ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 30)
    & (city_data_df["Max Temp"] > 25)
    & (city_data_df["Wind Speed"] < 10)
    & (city_data_df["Cloudiness"] == 0)
].dropna()

# Drop any rows with null values

# Display sample data
Ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
130,130,ratnagiri,16.9833,73.3000,27.66,43,0,2.24,IN,1735531093
182,182,tsiombe,-25.3000,45.4833,28.80,58,0,2.54,MG,1735531162
362,362,sabrum,23.0000,91.7319,25.90,36,0,2.11,IN,1735531399
406,406,pakokku,21.3333,95.1000,25.46,54,0,3.57,MM,1735531453
481,481,kailua-kona,19.6406,-155.9956,25.98,79,0,5.14,US,1735531553


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=Ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
130,ratnagiri,IN,16.9833,73.3000,43,
182,tsiombe,MG,-25.3000,45.4833,58,
362,sabrum,IN,23.0000,91.7319,36,
406,pakokku,MM,21.3333,95.1000,54,
481,kailua-kona,US,19.6406,-155.9956,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ratnagiri - nearest hotel: Vihar
tsiombe - nearest hotel: No hotel found
sabrum - nearest hotel: No hotel found
pakokku - nearest hotel: Shwe Pyi Hein
kailua-kona - nearest hotel: PACIFIC 19 Kona
kiama - nearest hotel: Nova Kiama


,City,Country,Lat,Lng,Humidity,Hotel Name
130,ratnagiri,IN,16.9833,73.3000,43,Vihar
182,tsiombe,MG,-25.3000,45.4833,58,No hotel found
362,sabrum,IN,23.0000,91.7319,36,No hotel found
406,pakokku,MM,21.3333,95.1000,54,Shwe Pyi Hein
481,kailua-kona,US,19.6406,-155.9956,79,PACIFIC 19 Kona
491,kiama,AU,-34.6833,150.8667,67,Nova Kiama


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:


# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "Hotel Name",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)